In [1]:
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224), # Applies Cropping
        transforms.RandomHorizontalFlip(), # Applies a Random Horizontal Flip
        transforms.ToTensor(), # Converts the image to a Tensor
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # Normalizes the Image
    ]),
    'val': transforms.Compose([
        transforms.Resize(256), #Applies Cropping
        transforms.CenterCrop(224), #
        transforms.ToTensor(), # Converts the image to a Tensor
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # Normalizes the Image
    ]),
}

data_dir = 'hymenoptera_data' # data directory

image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'val']}


dataloaders = {x: DataLoader(image_datasets[x], batch_size=4, shuffle=True, num_workers=4) for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes # Gets the all the classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

NameError: name 'transforms' is not defined

In [ ]:
def imshow(inp, title=None):
    """Display image for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0)) #transpose? 
    mean = np.array([0.485, 0.456, 0.406]) # mean of the image
    std = np.array([0.229, 0.224, 0.225]) # standard deviation for applying normalizing to the images
    inp = std * inp + mean # normalization
    inp = np.clip(inp, 0, 1) # any number less than 0 will become 0 and any number greater than 1 will become 1
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated


# Get a batch of training data
inputs, classes = next(iter(dataloaders['train']))

# Make a grid from batch
out = torchvision.utils.make_grid(inputs)

imshow(out, title=[class_names[x] for x in classes])

In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    # Create a temporary directory to save training checkpoints
    with TemporaryDirectory() as tempdir:
        best_model_params_path = os.path.join(tempdir, 'best_model_params.pt')

        torch.save(model.state_dict(), best_model_params_path)
        best_acc = 0.0

        for epoch in range(num_epochs):
            print(f'Epoch {epoch}/{num_epochs - 1}')
            print('-' * 10)

            # Each epoch has a training and validation phase
            for phase in ['train', 'val']:
                if phase == 'train':
                    model.train()  # Set model to training mode
                else:
                    model.eval()   # Set model to evaluate mode

                running_loss = 0.0
                running_corrects = 0

                # Iterate over data.
                for inputs, labels in dataloaders[phase]:
                    
                    # inputs the data into the gpu
                    inputs = inputs.to(device) 
                    labels = labels.to(device) 

                    # zero the parameter gradients
                    optimizer.zero_grad()

                    # forward
                    # track history if only in train
                    with torch.set_grad_enabled(phase == 'train'):
                        outputs = model(inputs)
                        _, preds = torch.max(outputs, 1)
                        loss = criterion(outputs, labels)

                        # backward + optimize only if in training phase
                        if phase == 'train':
                            loss.backward()
                            optimizer.step()

                    # statistics
                    running_loss += loss.item() * inputs.size(0) # fuck whats this.
                    
                    # total amount of predictions that equal to the actual label
                    running_corrects += torch.sum(preds == labels.data) 
                    
                if phase == 'train':
                    scheduler.step() 

                epoch_loss = running_loss / dataset_sizes[phase] # calculates the loss
                epoch_acc = running_corrects.double() / dataset_sizes[phase] # calculates the accuracy

                print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

                # deep copy the model
                if phase == 'val' and epoch_acc > best_acc:
                    best_acc = epoch_acc
                    torch.save(model.state_dict(), best_model_params_path)

            print()

        time_elapsed = time.time() - since
        print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
        print(f'Best val Acc: {best_acc:4f}')

        # load best model weights
        model.load_state_dict(torch.load(best_model_params_path)) # loads the model weights
    return model

In [ ]:
def visualize_model(model, num_images=20):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['val']):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                ax.set_title(f'predicted: {class_names[preds[j]]}')
                imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)

In [ ]:
model_ft = models.resnet18(weights='IMAGENET1K_V1')
num_ftrs = model_ft.fc.in_features
# Here the size of each output sample is set to 2.
# Alternatively, it can be generalized to ``nn.Linear(num_ftrs, len(class_names))``.

model_ft.fc = nn.Sequential(
#     nn.Dropout1d(0.4),
    nn.Linear(num_ftrs, 2048),
    nn.Linear(2048, 2)
)

# nn.Linear(num_ftrs, 2) # sets the fully connected layer to be just a Linear layer

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss() # loss function for classification

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9) # gradient descent with momentum

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1) 

In [2]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=25)

NameError: name 'train_model' is not defined

In [3]:
weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT # .DEFAULT = best available weights from pretraining on ImageNet
weights

NameError: name 'torchvision' is not defined

In [19]:
auto_transforms = weights.transforms()
auto_transforms

ImageClassification(
    crop_size=[224]
    resize_size=[256]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BICUBIC
)

In [20]:
eff_weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT # .DEFAULT = best available weights 
eff_net = torchvision.models.efficientnet_b0(weights=eff_weights).to(device)

In [21]:
summary(model=eff_net, 
        input_size=(32, 3, 224, 224), # make sure this is "input_size", not "input_shape"
        # col_names=["input_size"], # uncomment for smaller output
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]
)

Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
EfficientNet (EfficientNet)                                  [32, 3, 224, 224]    [32, 1000]           --                   True
├─Sequential (features)                                      [32, 3, 224, 224]    [32, 1280, 7, 7]     --                   True
│    └─Conv2dNormActivation (0)                              [32, 3, 224, 224]    [32, 32, 112, 112]   --                   True
│    │    └─Conv2d (0)                                       [32, 3, 224, 224]    [32, 32, 112, 112]   864                  True
│    │    └─BatchNorm2d (1)                                  [32, 32, 112, 112]   [32, 32, 112, 112]   64                   True
│    │    └─SiLU (2)                                         [32, 32, 112, 112]   [32, 32, 112, 112]   --                   --
│    └─Sequential (1)                                        [32, 32, 112, 112]   [32, 16, 112

In [22]:
!pip install torch torchvision torchaudio -f https://download.pytorch.org/whl/cu{CUDA_VERSION}/torch_stable.html

Looking in links: https://download.pytorch.org/whl/cu{CUDA_VERSION}/torch_stable.html
  Attempting uninstall: torch
    Found existing installation: torch 2.1.2
    Uninstalling torch-2.1.2:
      Successfully uninstalled torch-2.1.2


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\Brand\\anaconda3\\Lib\\site-packages\\~-rch\\lib\\asmjit.dll'
Consider using the `--user` option or check the permissions.



In [41]:
# If you are running this within a python venv and have the error "Module not found: numpy", 
# ensure that your kernel is set to $VENV_NAME and run $python -m ipykernel install --user --name=$VENV_NAME 
# You will need to pip install numpy, tqdm, vit_pytorch, and ipywidgets
import math
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.cuda as cuda
from torch.utils.data import DataLoader
import os
import time
import torchvision
from torchvision import datasets, transforms
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from torchinfo import summary
#import timm # Unofficial pytorch image models, for comparison
from PIL import Image
%matplotlib inline

from torchvision import datasets, models, transforms
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") # Use Nvidia GPU if available, for faster results
# Uncomment below to verify cuda is working
torch.cuda.is_available()
# print(torch.version.cuda)

False

In [29]:
# Define the transformations that will be applied to the images during the loading process
transform = transforms.Compose([
    transforms.Resize((128,128)),
    transforms.ToTensor(),
    #transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) # Mean and Std. Dev values used here are commonly used with the ImageNet dataset
])

In [30]:
# Training Settings
batch_size = 32 # This should usually be kept to a size that is a power of two
epochs = 15 # Need to monitor validation loss during training to avoid overfitting https://datascience.stackexchange.com/questions/46523/is-a-large-number-of-epochs-good-or-bad-idea-in-cnn
lr = 3e-5 # Need to implement learning rate decay 
gamma = 0.7
seed = 2147483647
# We'll use a PyTorch Generator to make things repeatable (deterministic)
g = torch.Generator().manual_seed(seed)

In [31]:
# Define path to dataset
dataset_path = "%s/../data/images/" % (os.getcwd())

# Load Datasets with labels
dset = datasets.ImageFolder(dataset_path, transform=transform) # Automatially assigns labels to examples based on the directory name

# Generate 2 splits: Train (80%), Test (20%)
dset_size = len(dset)
train_size = int(0.8 * dset_size)
test_size = dset_size - train_size

train, test = torch.utils.data.random_split(dset, [train_size, test_size], generator=g)

# Create Data Loaders fro splits
train_dl = DataLoader(train, batch_size=batch_size, shuffle=True)
test_dl = DataLoader(test, batch_size=batch_size, shuffle=True)

In [32]:
classes = len(dset.classes)

In [34]:
# Training Statistics Init
loss_dict = {}
accuracy_dict = {}
time_to_train_dict = {}
test_accuracy_dict = {}

def train_model(model, name):
    # Loss Function
    criterion = nn.CrossEntropyLoss()
    # Optimizer
    optimizer = optim.Adam(model.parameters(), lr=lr)
    # Scheduler
    scheduler = StepLR(optimizer, step_size=1, gamma=gamma)
    print("")
    print("++++++++++++++++++++++++++++++++++++++++")
    print(f"Training Run [Model: {name}]")
    print("++++++++++++++++++++++++++++++++++++++++")

    # Training Statistics Init
    loss_list = []
    accuracy_list = []
    
    # Training Time
#     start_event = cuda.Event(enable_timing=True)
#     end_event = cuda.Event(enable_timing=True)
    # Begin Clock
#     start_event.record()
    
    # Training Loop
    for epoch in range(epochs):
        epoch_loss = 0
        epoch_accuracy = 0
        
        for data, label in tqdm(train_dl):
            data = data.to(device) # Ensure we're processing data on GPU
            label = label.to(device)
        
            output = model(data)
            loss = criterion(output, label)
        
            optimizer.zero_grad() # Zero out the gradient -- we'll experience weird bugs if we forget to do so
            loss.backward()
            optimizer.step()
        
            acc = (output.argmax(dim=1) == label).float().mean()
            epoch_accuracy += acc / len(train_dl)
            epoch_loss += loss / len(train_dl)
        
        print(f"Epoch: {epoch+1} - loss: {epoch_loss:.4f} - acc: {epoch_accuracy:.4f}")
        loss_list.append(epoch_loss.cpu().detach().numpy().item())
        accuracy_list.append(epoch_accuracy.cpu().detach().numpy().item())
       
    # End Clock
#     end_event.record()
    cuda.synchronize() # Wait for GPU operations to complete
#     time = start_event.elapsed_time(end_event) / 1000 # Convert to seconds
    num_examples = batch_size * len(train_dl)
    time_per_example = time / (num_examples * epochs)
    print(f"It took {time} seconds to train {name} on {num_examples} examples over {epochs} epochs.")
    print(f"That averages to {time_per_example} seconds per example")

    # Test run
    print("++++++++++++++++++++++++++++++++++++++++")
    print(f"Test Run [Model: {name}] ")
    print("++++++++++++++++++++++++++++++++++++++++")
    accuracies = []
    batch_acc = 0
    for data, label in tqdm(test_dl):
        data = data.to(device)
        label = label.to(device)
        output = model(data)
        acc = (output.argmax(dim=1) == label).float().mean().cpu().detach().numpy()
        batch_acc += acc / len(test_dl)
        accuracies.append(batch_acc)
        
    print(f"Test Accuracy: {accuracies[-1]} - Number of test cases: {len(test_dl) * batch_size}")

    # Update training stats
    loss_dict.update({name: loss_list})
    accuracy_dict.update({name: accuracy_list})
    time_to_train_dict.update({name: time_per_example})
    test_accuracy_dict.update({name: accuracies[-1]})

In [35]:
# ResNet - 18
res_net_18 = models.resnet18(weights='IMAGENET1K_V1')
num_ftrs = res_net_18.fc.in_features
# Here the size of each output sample is set to 2.
# Alternatively, it can be generalized to ``nn.Linear(num_ftrs, len(class_names))``.

In [36]:
for param in res_net_18.parameters():
    param.requires_grad = False

In [37]:
for param in res_net_18.fc.parameters():
    param.requires_grad = True

In [38]:
res_net_18.fc = nn.Sequential(
    nn.Linear(num_ftrs, classes)
)

In [39]:
train_model(res_net_18, "ResNet 18")


++++++++++++++++++++++++++++++++++++++++
Training Run [Model: ResNet 18]
++++++++++++++++++++++++++++++++++++++++


  0%|          | 0/294 [00:00<?, ?it/s]

Epoch: 1 - loss: 2.1980 - acc: 0.2918


  0%|          | 0/294 [00:00<?, ?it/s]

Epoch: 2 - loss: 1.8690 - acc: 0.3720


...

  0%|          | 0/294 [00:00<?, ?it/s]

Epoch: 14 - loss: 1.5091 - acc: 0.4671


  0%|          | 0/294 [00:00<?, ?it/s]

Epoch: 15 - loss: 1.4973 - acc: 0.4678


NameError: name 'end_event' is not defined

In [43]:
summary(model=res_net_18, 
        input_size=(32, 3, 128, 128), # make sure this is "input_size", not "input_shape"
        # col_names=["input_size"], # uncomment for smaller output
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]
)

Layer (type (var_name))                  Input Shape          Output Shape         Param #              Trainable
ResNet (ResNet)                          [32, 3, 128, 128]    [32, 23]             --                   Partial
├─Conv2d (conv1)                         [32, 3, 128, 128]    [32, 64, 64, 64]     (9,408)              False
├─BatchNorm2d (bn1)                      [32, 64, 64, 64]     [32, 64, 64, 64]     (128)                False
├─ReLU (relu)                            [32, 64, 64, 64]     [32, 64, 64, 64]     --                   --
├─MaxPool2d (maxpool)                    [32, 64, 64, 64]     [32, 64, 32, 32]     --                   --
├─Sequential (layer1)                    [32, 64, 32, 32]     [32, 64, 32, 32]     --                   False
│    └─BasicBlock (0)                    [32, 64, 32, 32]     [32, 64, 32, 32]     --                   False
│    │    └─Conv2d (conv1)               [32, 64, 32, 32]     [32, 64, 32, 32]     (36,864)             False
│    │    

In [53]:
# Resnet - 50
res_weights = torchvision.models.ResNet50_Weights.DEFAULT
res_net_50 = torchvision.models.resnet50(weights=res_weights).to(device)
num_ftrs = res_net_50.fc.in_features
# Here the size of each output sample is set to 2.
# Alternatively, it can be generalized to ``nn.Linear(num_ftrs, len(class_names))``.

num_ftrs

2048

In [54]:
for param in res_net50.parameters():
    param.requires_grad = False

for param in res_net50.fc.parameters():
    param.requires_grad = True

res_net50.fc = nn.Sequential(
    nn.Linear(num_ftrs, classes)
)

train_model(res_net50, "ResNet 50")


++++++++++++++++++++++++++++++++++++++++
Training Run [Model: ResNet 50]
++++++++++++++++++++++++++++++++++++++++


  0%|          | 0/294 [00:00<?, ?it/s]

Epoch: 1 - loss: 2.4336 - acc: 0.3475


  0%|          | 0/294 [00:00<?, ?it/s]

Epoch: 2 - loss: 1.8860 - acc: 0.4120


...

  0%|          | 0/294 [00:00<?, ?it/s]

Epoch: 14 - loss: 1.4347 - acc: 0.5087


  0%|          | 0/294 [00:00<?, ?it/s]

Epoch: 15 - loss: 1.4210 - acc: 0.5077


NameError: name 'end_event' is not defined

In [52]:
summary(model=res_net50, 
        input_size=(32, 3, 128, 128), # make sure this is "input_size", not "input_shape"
        # col_names=["input_size"], # uncomment for smaller output
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]
)

Layer (type (var_name))                  Input Shape          Output Shape         Param #              Trainable
ResNet (ResNet)                          [32, 3, 128, 128]    [32, 1000]           --                   False
├─Conv2d (conv1)                         [32, 3, 128, 128]    [32, 64, 64, 64]     (9,408)              False
├─BatchNorm2d (bn1)                      [32, 64, 64, 64]     [32, 64, 64, 64]     (128)                False
├─ReLU (relu)                            [32, 64, 64, 64]     [32, 64, 64, 64]     --                   --
├─MaxPool2d (maxpool)                    [32, 64, 64, 64]     [32, 64, 32, 32]     --                   --
├─Sequential (layer1)                    [32, 64, 32, 32]     [32, 256, 32, 32]    --                   False
│    └─Bottleneck (0)                    [32, 64, 32, 32]     [32, 256, 32, 32]    --                   False
│    │    └─Conv2d (conv1)               [32, 64, 32, 32]     [32, 64, 32, 32]     (4,096)              False
│    │    └─

In [62]:
# VGG - 16 no Batch Normalization
weights = torchvision.models.VGG16_Weights.DEFAULT
vggnet_16 = torchvision.models.vgg16(weights=weights).to(device)
# num_ftrs = res_net_50.fc.in_features

for param in vggnet_16.parameters():
    param.requires_grad = False

for param in vggnet_16.classifier.parameters():
    param.requires_grad = True

vggnet_16.classifier[6] = nn.Linear(4096, classes)

train_model(vggnet_16, "VGG 16")


++++++++++++++++++++++++++++++++++++++++
Training Run [Model: VGG 16]
++++++++++++++++++++++++++++++++++++++++


  0%|          | 0/294 [00:00<?, ?it/s]

Epoch: 1 - loss: 1.6452 - acc: 0.4361


  0%|          | 0/294 [00:00<?, ?it/s]

Epoch: 2 - loss: 1.3824 - acc: 0.5078


...

Epoch: 4 - loss: 1.0847 - acc: 0.6133


  0%|          | 0/294 [00:00<?, ?it/s]

Epoch: 5 - loss: 0.9172 - acc: 0.6770


  0%|          | 0/294 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
summary(model=vggnet_16, 
        input_size=(32, 3, 128, 128), # make sure this is "input_size", not "input_shape"
        # col_names=["input_size"], # uncomment for smaller output
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]
)

In [61]:
# VGG - 16 with Batch Normalization

weights = torchvision.models.VGG16_BN_Weights.DEFAULT
vggnet_16_bn = torchvision.models.vgg16_bn(weights=weights).to(device)
# num_ftrs = vggnet_16_bn.fc.in_features

for param in vggnet_16.parameters():
    param.requires_grad = False

for param in vggnet_16.classifier.parameters():
    param.requires_grad = True

vggnet_16.classifier[6] = nn.Linear(4096, classes)

# train_model(vggnet_16, "VGG 16 with Batch Normalization")

Layer (type (var_name))                  Input Shape          Output Shape         Param #              Trainable
VGG (VGG)                                [32, 3, 128, 128]    [32, 23]             --                   Partial
├─Sequential (features)                  [32, 3, 128, 128]    [32, 512, 4, 4]      --                   False
│    └─Conv2d (0)                        [32, 3, 128, 128]    [32, 64, 128, 128]   (1,792)              False
│    └─ReLU (1)                          [32, 64, 128, 128]   [32, 64, 128, 128]   --                   --
│    └─Conv2d (2)                        [32, 64, 128, 128]   [32, 64, 128, 128]   (36,928)             False
│    └─ReLU (3)                          [32, 64, 128, 128]   [32, 64, 128, 128]   --                   --
│    └─MaxPool2d (4)                     [32, 64, 128, 128]   [32, 64, 64, 64]     --                   --
│    └─Conv2d (5)                        [32, 64, 64, 64]     [32, 128, 64, 64]    (73,856)             False
│    └─ReLU (

In [ ]:
summary(model=vggnet_16, 
        input_size=(32, 3, 128, 128), # make sure this is "input_size", not "input_shape"
        # col_names=["input_size"], # uncomment for smaller output
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]
)

In [ ]:
# Mobile Net Small Params
weights = torchvision.models.MobileNet_V3_Small_Weights.DEFAULT
mobile_net = torchvision.models.mobilenet_v3_small(weights=res_weights).to(device)
num_ftrs = mobile_net.fc.in_features

for param in mobile_net.parameters():
    param.requires_grad = False

for param in mobile_net.fc.parameters():
    param.requires_grad = True

mobile_net.fc = nn.Sequential(
    nn.Linear(num_ftrs, classes)
)

train_model(mobile_net, "Mobile Net Small Weights")

In [ ]:
summary(model=mobile_net, 
        input_size=(32, 3, 128, 128), # make sure this is "input_size", not "input_shape"
        # col_names=["input_size"], # uncomment for smaller output
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]
)

In [ ]:
# Mobile Net Large Params
weights = torchvision.models.MobileNet_V3_Large_Weights.DEFAULT
mobile_net_large = torchvision.models.resnet50(weights=res_weights).to(device)
num_ftrs = mobile_net_large.fc.in_features

for param in mobile_net_large.parameters():
    param.requires_grad = False

for param in mobile_net_large.fc.parameters():
    param.requires_grad = True

mobile_net_large.fc = nn.Sequential(
    nn.Linear(num_ftrs, classes)
)

train_model(mobile_net_large, "ResNet 50")

In [ ]:
summary(model=vggnet_16, 
        input_size=(32, 3, 128, 128), # make sure this is "input_size", not "input_shape"
        # col_names=["input_size"], # uncomment for smaller output
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]
)

In [ ]:
# Alex Net
weights = torchvision.models.AlexNet_Weights.DEFAULT
alex_net = torchvision.models.AlexNet_Weights(weights=weights).to(device)
num_ftrs = alex_net.fc.in_features

for param in alex_net.parameters():
    param.requires_grad = False

for param in alex_net.fc.parameters():
    param.requires_grad = True

alex_net.fc = nn.Sequential(
    nn.Linear(num_ftrs, classes)
)

train_model(alex_net, "Alex Net")

In [ ]:
summary(model=alex_net, 
        input_size=(32, 3, 128, 128), # make sure this is "input_size", not "input_shape"
        # col_names=["input_size"], # uncomment for smaller output
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]
)

In [ ]:
# VGG 19
weights = torchvision.models.VGG19_Weights.DEFAULT
vgg_19_net = torchvision.models.vgg19(weights=weights).to(device)
num_ftrs = vgg_19_net.fc.in_features

for param in vgg_19_net.parameters():
    param.requires_grad = False

for param in vgg_19_net.fc.parameters():
    param.requires_grad = True

vgg_19.fc = nn.Sequential(
    nn.Linear(num_ftrs, classes)
)

train_model(vgg_19_net, "VGG 19")

In [ ]:
summary(model=vgg_19_net, 
        input_size=(32, 3, 128, 128), # make sure this is "input_size", not "input_shape"
        # col_names=["input_size"], # uncomment for smaller output
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]
)